In [1]:
import pandas as pd

In [2]:
import os

In [3]:
os.getcwd()

'/Users/sophieyun/OneDrive - UCLA IT Services/Grad School/Research/Hongjing/social intention similarity'

In [4]:
cd HSvideo-similarity/ratingLabeling/tempData

/Users/sophieyun/OneDrive - UCLA IT Services/Grad School/Research/Hongjing/social intention similarity/HSvideo-similarity/ratingLabeling/tempData


In [5]:
dat = pd.read_csv("48subjCleanedTrial_HSvideo.csv")
dat.head()

,subjNum,subjStartDate,subjStartTime,trialIndex,exptId,rating,rtRating,vidPlayCountsRating,label,labelIndex,comment,rtLabel,rtComment,vidPlayCountsLabel,vidPlayCountsComment
0,2,2023-06-06,15:41:31,0,4408_lead,4,14.401,1,undefined,undefined,undefined,undefined,undefined,undefined,undefined
1,2,2023-06-06,15:41:31,1,5814_talk to,1,12.605,1,undefined,undefined,undefined,undefined,undefined,undefined,undefined
2,2,2023-06-06,15:41:31,2,5843_huddle with,5,13.244,1,undefined,undefined,undefined,undefined,undefined,undefined,undefined
3,2,2023-06-06,15:41:31,3,4397_hug,5,14.755,1,undefined,undefined,undefined,undefined,undefined,undefined,undefined
4,2,2023-06-06,15:41:31,4,5902_hit,4,13.797,1,undefined,undefined,undefined,undefined,undefined,undefined,undefined


In [8]:
sum(dat.label != "undefined") #1296 labeling answers in total

1296

In [13]:
labelDat = dat.loc[dat.label != "undefined", ["subjNum", "exptId", "label"]]

### Drop participants who chose the same label in more than half of the videos

In [18]:
videoNum = 27
dropNum = round(videoNum/2) #drop this participant if they have choose same label more than in this many trials

In [29]:
sum(labelDat.groupby("subjNum").apply(lambda subj: subj["label"].value_counts()[0] >= dropNum)) #no one to drop


0

## Create label summary matrix

In [30]:
labelDat

,subjNum,exptId,label
27,2,4408_lead,Poke
28,2,5814_talk to,Ignore
29,2,5843_huddle with,Huddle with
30,2,4397_hug,Flirt with
31,2,5902_hit,Poke
...,...,...,...
2587,64,5870_poke,Flirt with
2588,64,6017_encircle,Encircle
2589,64,6005_throw,Fight
2590,64,5878_follow,Lead


In [41]:
shortLabel = labelDat.label.value_counts().keys()

In [71]:
shortLabel = shortLabel.sort_values()

In [72]:
shortLabel

Index(['Accompany', 'Approach', 'Avoid', 'Bother', 'Capture', 'Creep up on',
       'Encircle', 'Escape', 'Examine', 'Fight', 'Flirt with', 'Follow',
       'Herd', 'Hit', 'Huddle with', 'Hug', 'Ignore', 'Kiss', 'Lead', 'Leave',
       'Poke', 'Pull', 'Push', 'Scratch', 'Talk to', 'Throw', 'Tickle'],
      dtype='object')

In [54]:
longLabel = labelDat.exptId.value_counts().keys()

In [63]:
longLabelDf = pd.DataFrame([action.split("_") for action in longLabel])

In [65]:
longLabelDf

,0,1
0,4408,lead
1,6034,avoid
2,6005,throw
3,5991,examine
4,5870,poke
5,6012,kiss
6,6016,scratch
7,5809,pull
8,5948,flirt with
9,1012,push


In [68]:
longLabelDf = longLabelDf.sort_values(by=[1])

In [69]:
longLabelDf

,0,1
21,5787,accompany
15,5849,approach
1,6034,avoid
19,6079,bother
20,6035,capture
12,5987,creep up on
13,6017,encircle
22,5875,escape
3,5991,examine
18,5986,fight


In [76]:
longLabel = longLabelDf.apply(lambda row: row[0] + "_" + row[1], axis = 1)

In [78]:
label = labelDat["label"].replace(shortLabel, longLabel)

In [79]:
label

27             5870_poke
28           5816_ignore
29      5843_huddle with
30       5948_flirt with
31             5870_poke
              ...       
2587     5948_flirt with
2588       6017_encircle
2589          5986_fight
2590           4408_lead
2591        6016_scratch
Name: label, Length: 1296, dtype: object

In [81]:
labelDat["longLabel"] = label

In [83]:
labelDat

,subjNum,exptId,label,longLabel
27,2,4408_lead,Poke,5870_poke
28,2,5814_talk to,Ignore,5816_ignore
29,2,5843_huddle with,Huddle with,5843_huddle with
30,2,4397_hug,Flirt with,5948_flirt with
31,2,5902_hit,Poke,5870_poke
...,...,...,...,...
2587,64,5870_poke,Flirt with,5948_flirt with
2588,64,6017_encircle,Encircle,6017_encircle
2589,64,6005_throw,Fight,5986_fight
2590,64,5878_follow,Lead,4408_lead


### Long to Wide dataframe

In [87]:
wideLabelDat = pd.pivot(labelDat, index="subjNum", columns="exptId", values="label")

In [88]:
wideLabelDat.to_excel("48subjLabelingData.xlsx")